# Simple Arb Bot Project Report
Wanli Zhou         Aug 17 2024

## I. Project Description

## II. Background and Motivation

Arbitrage is a trading strategy that involves taking advantage of price discrepancies of the same asset in different markets or forms to make a profit without risk. The core idea is to buy low in one market and sell high in another simultaneously, thereby capitalizing on the price difference. 

With the rise of blockchain technology, decentralized exchanges (DEXes) have seen tremendous growth. The top 10 DEXs, including Uniswap, Curve, and PancakeSwap, had a combined trading volume of $66.6 billion in Q2 2023. The price discrepancies of assets across these different DEX markets thus become huge arbitrage opportunities that I'm interested in exploring.

This project aims to develop an automated arbitrage bot for Uniswap V2 DEXes and profit from the price differences of the same asset in different DEX markets. This bot will collect live price of a list of assets from on-chain smart contracts, and make informed decisions about when to execute simultaneous arbitrage transactions. The ultimate goal is to profit from these transactions while contributing to the overall efficiency and balance of DeFi markets by aligning prices across different markets or exchanges. 

### How Arbitrage Works in Decentralized Exchanges
1. **Identification of Price Discrepancy**: The trader identifies an asset that is priced differently in two or more markets.
2. **Simultaneous Transactions**: The trader buys the asset at the lower price in one market and sells it at the higher price in another market simultaneously.
3. **Profit Realization**: The profit is the difference between the buying and selling prices, minus any transaction costs such as blockchain gas fees and exchange costs.

### Major Types of On-Chain DEXes
On-chain exchanges are decentralized platforms that facilitate the trading of cryptocurrencies directly on the blockchain. The two major types of on-chain exchanges are Central Limit Order Book (CLOB) exchanges and Automated Market Maker (AMM) exchanges.

1. Central Limit Order Book (CLOB) Exchanges operate similarly to traditional financial exchanges. They maintain an order book that lists all the buy and sell orders for a particular asset, and trades are executed when buy and sell orders match.
   
   Key features of a CLOB exchange include:
	- **Order Book**: A ledger where all buy and sell orders are recorded.
	  
	  <img src="images/Orderbook.png" alt="ETH-USD Perpetual's Orderbook on dYdX Exchange" width="700"/>
	
	- **Order Matching**: Orders are matched based on price and time priority.
	  
	- **Trade Execution**: Trades occur when buy and sell orders match.
	  
	- **Liquidity Providers**: Users place orders in the order book, providing liquidity.
   

2. Automated Market Maker (AMM) Exchanges use smart contracts to create liquidity pools, where users can trade against the pool rather than matching with another user. Prices are determined by mathematical formulas.

   Key features of an AMM exchange include:
	- **Liquidity Pools**: Pools of tokens provided by liquidity providers (LPs). Each liquidity pool is a trading venue for a pair of tokens.

	  <img src="images/AMM_pool_by_Uniswap.png" alt="Concept of an AMM pool by Uniswap" width="700"/><br>
	
	- **Pricing Algorithm**: Prices are determined by algorithms like the constant product formula (x * y = k).

	  <img src="images/AMM_Swap_by_Uniswap.png" alt="Concept of an AMM Swap by Uniswap" width="700"/><br>
	
	- **Liquidity Providers**: Users provide liquidity to pools and earn fees.
	
<br>
Since prices are discovered in a different way on each model, this creates many arbitrage opportunities between **CLOB x AMM** or even between two different AMMs.

### Factors in a Profitable Arbitrage Transaction
1. **Slippage**<br>
   Slippage refers to the difference between the expected price of a trade and the actual price at which the trade is executed. In the context of an arbitrage trade, slippage refers to the price change between the time an arbitrage opportunity is spotted and the time the arbitrage trade is finalized on-chain. Slippage can occur due to various factors, including low liquidity, high volatility, or delays in transaction processing.
   
   - Liquidity: <br>
      AMMs rely on liquidity pools to facilitate trades. If a liquidity pool has a small amount of an asset, large trades can significantly impact the price, leading to higher slippage. In contrast, larger liquidity pools can absorb bigger trades with minimal price impact, reducing slippage.
     
   - Volatility: <br>
      High volatility in the market can lead to rapid price changes, increasing the likelihood of slippage, increasing the likelihood of slippage.
     
   - Blockchain transaction speed: <br>
      Due to the nature of blockchain, every transaction on-chain participates in an auction, where all users compete to see who will pay more to have their transactions validated first on the blockchain. Users can set a **gas price** for their transaction, where a higher gas price means higher resulting gas cost, but faster processing time of their transaction; a lower gas price means saving on transaction costs, but risking too long to complete the transaction and losing the arbitrage opportunity. Thus, an arbitrageur must always be looking for the best blockchain transaction price, balancing cost and speed.
<br>
<br>

1. **Concurrency**<br>
   Trades must be executed as fast as possible to capture the price difference opportunity of an asset. This means that funds will need to be deposited on both wallets for the different markets so that the bot can do both the buy and sell transactions at the same time.
<br>
<br>

1. **Costs**<br>   
   Profitability of an arbitrage trade also needs to factor in the blockchain gas cost, and the exchange transaction fees.

## III. Product Design

### Architecture Design

### Strategy Design
With the profitability factors listed above, my strategy will be formulated around the following design principles:
1. An arbitrage trade will only be executed when the $\text{price difference(\%)}$ based on a user-defined $\text{minimum profitability(\%)}$ is reached.
   
2.  $\text{price difference(\%)}$ is calculated using the following formula:
   $$\text{min. profitability}+\frac{(\text{gas costs + trading fees + revert overhead})}{\text{order value}}+\text{slippage buffer}$$

3. The $\text{revert overhead}$ is calculated using the following formula:
   $$\text{avg. gas cost per reverted transaction} * \text{revert rate up to the last transaction}$$

4. The $\text{slippage buffer}$ is a user-defined percentage enforced on AMM trades, which will fail a trade if the final slippage exceeds the pre-defined percentage.

5. Trades in one arbitrage transaction will be executed only when both trades can be executed under predefined $\text{minimum profitability}$ and $\text{slippage buffer}$, through smart contract enforcement. If any condition causes at least one trade to fail, the transaction is reverted, ensuring that neither trade is executed.


### Trading Environment Simulation
As the local Ethereum environment is forked from a fixed state, the project uses procedurally generated transactions of random trades to create a trading environment to test and optimize the performance of the strategy implemented.

In order to mimic a realistic trading environment with different trader profiles and liquidity operation scenarios, four different trade simulation bots are created in `trading_env_sims`. Specifically,

1. `trading_env_sims/whale_trading.py`: a whale trader simulator that trades ETH (the gas token on Ethereum) for ERC20 (other non-gas assets) on UniswapV2.
    - The whale account holds ~$300k worth of each baseAsset listed in `configs/mainnet.json`.
    - Every 5 seconds, the whale trades ETH for another random baseAsset for a random value between $30,000 and $100,000 in a single swap.
<br>
<br>

1. `trading_env_sims/whale_erc20_trading.py` a whale trader simulator that trades ERC20 for ERC20 on UniswapV2:
    - The whale account holds ~$300k worth of each baseAsset listed in `configs/mainnet.json`.
    - Every 5 seconds, the whale trades a random baseAsset for another random baseAsset for a random value between $30,000 and $100,000 in a single swap.
<br>
<br>

1. `trading_env_sims/regular_trading.py` a regular trader simulator that trades ETH for ERC20 on SushiSwap. 
    - The regular trader account holds ~$15k worth of each baseAsset listed in `configs/mainnet.json`.
    - Every second, the regular trader trades ETH for another random baseAsset for a random value between $300 and $9,000 in a single swap.
<br>
<br>

1. `trading_env_sims/regular_erc20_trading.py` a regular trader simulator that trades ERC20 for ERC20 on SushiSwap. 
    - The regular trader account holds ~$15k worth of each baseAsset listed in `configs/mainnet.json`.
    - Every second, the regular trader trades a random baseAsset for another random baseAsset for a random value between $300 and $9,000 in a single swap.
<br>
<br>

The trade simulation bots are run through the shell script `./scripts/simulate_trade_env.sh`.

For future interations, more types of trade simulation bots will be created, covering profiles including:
    - liquidity providers who add and withdraw liquidity
    - competing arbitrage bot

## IV. Code Design UML(before implementation, for reference only)
![Simple Arbitrage Bot Architecture Design UML](images/Simple_Arb_Bot_Architecture_Design.png)

## V. Code Implementation 
GitHub link:
https://github.com/bigbowlz/simple_arb_bot

Please follow the README.md to set up a local trade environment and start a bot instance.

## VI. Bot Performance Evaluation

### Performance Metrics
1. Profitability
    - Net Profit: Calculate the total profits after deducting all costs, including transaction fees, gas fees, and any other operational costs. Net Profit = BalanceAfter - BalanceBefore.
    - Return on Investment (ROI): ROI = (Net Profit / Total Investment) * 100%. This measures the efficiency of the investment. Specifically, total investment refers to all liquidity provided to the arb contract as well as gas. In the actual implementation, as gas is supplied from a test account with unlimited ETH and is unable to factor in, it is omitted in the ROI calculation.
    - Profit per Trade: Average profit per arbitrage opportunity.
2. Success Rate
    - Winning Trades vs. All Trades: The ratio of profitable trades to all trades. Losing trades include both trades that execute on exchanges with a net loss, and trades that fail to execute on exchanges but incur gas costs.
3. Execution Speed
    - Analysis Latency: Time taken to detect and execute an arbitrage opportunity.  
    - On-chain Execution Time: The time between placing the order and the order getting executed, which reflects gas price competitiveness and serves gas optimization reference.
4. Capital Utilization
    - Trade Volume: The total volume of trades executed by the bot.
	- Capital Efficiency (not included in the current implementation): Percentage of available capital actively used in arbitrage opportunities, calculated with averageLiquidity/totalLiquidity. This does not factor in gas. This metric is not included in the demo as the account balances are arbitrary due to a test environment.


### Performance Experiment
To evaluate the performance of the bot, I created 3 bot runs with the same configurations, and recorded the bot performance respectively for an average overview of the overall performance of the arbitrage bot implementation.

The results of each run is located in the `experiment_logs` directory, under their respective folder.

The three runs were conducted with the exact same configurations as below, which can be found in the `arb_bot_config.json` file.
- 500 bps in minimum profit
- 1000 bps in slippage buffer
- and 30 minutes in run time

The experiement 

### Industry Performance (not benchmarked)
Due to the business nature of arbitrage bots, there are no open-source implementations that I can find for *production* uses. Thus all of the following performance benchmarks/references are obtained from communications (social media posts or blog announcements) of some arbitrage bots, rather than from actual benchmarking in the same test environment.
1. Profitability
2. Execution Speed

## VII. Toolings and Implementation References

#### Smart Contract development and interactions
**web3.py** - a Python library for interacting with Ethereum.

web3.py is used in the project to help with sending transactions, interacting with smart contracts, reading block data and transaction data, and other utilities functions including converting currencies.

**Hardhat** -  a development environment for Ethereum software. Compile, test, debug and deploy smart contracts. Supporting network forking to simulate real data in a test environment. Tutorial: https://hardhat.org/tutorial

Hardhat is used in the project to host a locally run Ethereum mainnet fork and deploy the arbitrage contract.


#### Existing bot implementations:
HummingBot: open-source framework that helps you design, backtest, and deploy automated trading bots. https://hummingbot.org/

An eductational implementation: https://github.com/jamesbachini/DEX-Arbitrage

